# Mapping Uncertainty using Ensemble Modelling
### Abstract
>When modelling physical systems it is sometimes useful to understand how uncertainty varies in the domain (raneg of input variable values) of the system. One way of doing this is to build a representation of the uncertainty either at the full dimensionality of the domain or aggregation of the uncertainty into a lower dimensional subdomain. A model of the system may consist of a noise-free model of the expected average behaviour with a model of the residual noise added. Thus a simulation of the model might be $p = p_m + p_n$ where $p$ is the simulated property values, $p_m$ is the modelled part of the simulation and $p_n$ is a simulation of the noise model.
>
>The uncertainty in the model can be expressed using maps descriptive statastics of the model in the domain. Useful statistics include the mean, variance, standard deviation and relative standard deviation (RSD). To obtain these descriptive statistics it is necessary to produce a sampling of the models and calculating statistics from these. Given that the simulated value is the sum of the nose free and noise models it is possible to compute the mean and variance of the noise free $\mu_m$ and $\sigma_m^2$ and for the noise $\mu_n$ and $\sigma_n^2$ independently and combine them to produce the mean $\mu$ and variance $\sigma^2$ for the simulations. Other statistics can be derived from the mean and variance using the definitions for standard deviation $\sigma = \sqrt{\sigma^2}$ and for $RSD = \frac{\sigma}{\mu}$
>
>The case where the uncertainty for an ensemble of noise free models is combined with a noise model based on [*Gaussian Process*](https://en.wikipedia.org/wiki/Gaussian_process) is considered.

## Introduction
This is a study of how to map the uncertainty of a modelling process. For the purposes of this study a single property will be modelled, thus the model will be a function from a vector $\mathbf{x}$ to a single estimated value $\hat{p}$, thus $\hat{p} = \mathrm{f}\left(\mathbf{x}\right): \mathbb{R}^n \mapsto \mathbb{R}$. A map of the uncertainty is the distribution of the uncertainty in the domain (or a subdomain) of the model, for variance it is a map of the variance as a function of the domain $\sigma^2\left(\mathbf{x^\prime}\right) = \mathbb{E}\left[\left(\mathrm{f}\left(\mathbf{x^\prime}\right) - \mathbb{E}\left[\mathrm{f}\left(\mathbf{x^\prime}\right)\right]\right)^2\right] : \mathbb{R}^m \mapsto \mathbb{R}$ where the expectation is over all possible models $\mathrm{f(x^\prime)}$, where $m \le n$, if $m \lt n$ the map will be an aggragation of the uncertainty in the missing dimensions into a subdomain.

It is assumed that the modelling uses some data $\mathbf{d}$ to which models can be fitted. The fitting process will produce multiple, possible models rather than a single "best case" model. After fitting there will multiple residuals given $\mathbf{r_i} = \mathbf{d} - \mathbf{p_{m,i}}$ where $\mathbf{r_i}$ are the residuals for model $i$ calculated by subtracting the predicted values for a model $\mathbf{p_{m,i}}$ from the observed data set. Analysis of each set of residuals will produce its own model of the noise.

In this study the residual noise will be modelled using Gaussian Process. Gaussian Processes can be ["completely defined by their second-order statistics"](https://en.wikipedia.org/wiki/Gaussian_process#Covariance_functions), this means that only the variance (and co-variance) needs to be modelled. The model is typically a function which describes the variance as a function of separation. A feature of Gaussian Process is that it is possible to estimate the mean and variance from the model without the need to sample model simulations.

## Case Study: Uncertainty Map for a 3D Property
As an example, a 3 dimensional property will be studied. The simulations will occur on a regular grid of cells indexed as $ijk$. For the data available a number $n_m$ of models were produced, for each model a model of the noise is made and used to generate a large number $N$ simulations. Thus for a single model and simulation of the noise for a cell we have.
$$
p_{ijk} = p_{m,ijk} + p_{n,ijk}
$$
Where $p_{ijk}$ is the simulation value for cell $ijk$, $P_{m,ijk}$ is the value predicted by the model for cell $ijk$ and $P_{n,ijk}$ is value for the noise for cell $ijk$.

The 3D simulations of the property can be expressed as a vector, so the equation above for a property model $i$ combined with a simulation $j$ of the noise can be expressed as.
$$
\mathbf{p_{i,j}} = \mathbf{p_{m,i}} + \mathbf{p_{n,i,j}}
$$

To map the uncertainty the mean and variance can be calculated by sampling the population of property models combined with simulations of the noise. If the sample consists of $n_m$ property models and each is combined with $N$ simulations of the noise. The mean is given by.
$$
\overline{\mathbf{p}} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{m,i}} + \mathbf{p_{n,i,j}}\right)}{n_mN}
$$
and the sample variance will be given by
$$
s_\mathbf{p}^2 = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\left(\mathbf{p_{m,i}} + \mathbf{p_{n,i,j}}\right) - \overline{\mathbf{p}}\right)^2}{n_mN - 1}
$$

The equation for the mean value can be rearranged to give.
$$
\overline{\mathbf{p}} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \mathbf{p_{m,i}}}{n_mN} +
\frac{\sum_{i=1}^{n_m} \frac{1}{N}\sum_{j=1}^N \mathbf{p_{n,i,j}}}{n_m}
$$

The term $ \frac{1}{N}\sum_{j=1}^N \mathbf{p_{n,i,j}}$ is the mean of the noise models for property model $i$, substituting this term with $\overline{p_{n,i}}$ gives.
$$
\overline{\mathbf{p}} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \mathbf{p_{m,i}}}{n_mN} +
\frac{\sum_{i=1}^{n_m} \overline{\mathbf{p_{n,i}}}}{n_m}
$$

The two terms are just the mean of their respective values for all property models.
$$
\overline{\mathbf{p}} = \overline{\mathbf{p_m}} + \overline{\overline{\mathbf{p_n}}}
$$

A feature of Guassian Process is that $\overline{\mathbf{p_{n,i}}}$ can be calculated from the noise model without the need to run simulations of the model. So the calulation of the map of the mean values can be achieved by summing the map of the mean value of the noise-free property models and the mean of Gaussian Process interpolation for each of the property models.

This expression for the mean value of the property can be substituted into the equation for the sample variance, after collecting together similar terms gives.

$$
\mathbf{s_p^2} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\left(\mathbf{p_{m,i}} - \overline{\mathbf{p_m}}\right) + 
\left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right) \right)^2}{n_mN - 1}
$$

Expansion of the square and distributing the $\frac{1}{n_mN - 1}$ over the three terms gives.
$$
\mathbf{s_p^2} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{m,i}} - \overline{\mathbf{p_m}}\right)^2}{n_mN - 1} + \frac{2\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{m,i}} - \overline{\mathbf{p_m}}\right) \left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right)}{n_mN - 1} + \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right)^2}{n_mN - 1}
$$

The middle term is the covariance between the property models and the noise simulations. Since the noise simulations sample normally distributed random variables $\mathcal{N}\left(\mu, \sigma\right)$ it is not covariant with the property mdoels and so the middle term can be dropped.
$$
\mathbf{s_p^2} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{m,i}} - \overline{\mathbf{p_m}}\right)^2}{n_mN - 1} + \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right)^2}{n_mN - 1}
$$

In $\lim_{N \to\infty}$ then the denominator term reduces to $n_mN$.
$$
\mathbf{s_p^2} = \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{m,i}} - \overline{\mathbf{p_m}}\right)^2}{n_mN} + \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right)^2}{n_mN}
$$

In the first term the property models are constant for all noise simulations so after cancellation of the factor $N$ it can be simplified to $\frac{\sum_{i=1}^{n_m} \left(\mathbf{p_{m,i}} - \overline{\mathbf{p_m}}\right)^2}{n_m}$ which is the population variance of the property over all property models $\mathbf{\sigma_{p_m}^2}$.
$$
\mathbf{s_p^2} =\mathbf{\sigma_{p_m}^2} + \frac{\sum_{i=1}^{n_m} \sum_{j=1}^N \left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right)^2}{n_mN}
$$

Rearranging the second term gives $\frac{\sum_{i=1}^{n_m} \frac{1}{N}\sum_{j=1}^N \left(\mathbf{p_{n,i,j}} - \overline{\mathbf{p_{n,i}}}\right)^2}{n_m}$, this is simply the mean value of the variances for the noise models over all property models $\overline{\mathbf{\sigma_{p_n}^2}}$.
$$
\mathbf{s_p^2} = \mathbf{\sigma_{p_m}^2} + \overline{\mathbf{\sigma_{p_n}^2}}
$$

A feature of Guassian Process is that the variance of the noise for a property model can be calculated from the noise model with running simulations, so the $\mathbf{\sigma_{p_{n,i}}^2}$ can be calculated directly from the noise model and averaged separately of $\mathbf{\sigma_{p_m}^2}$.